In [366]:
import numpy as np
import random
from random import randint
from gurobipy import *

In [367]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

In [368]:
# Product Demand
demand = np.random.randint(0,20,(Products,Market))

In [369]:
# Cost of opening
f_i = [20, 25]
f_j = [15, 5, 10]

In [370]:
# Unit cost of manufacturing product 
Manufacturing_costs = np.random.uniform(0,5, (Manufacturing_plants,Products))

In [371]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,5, (Products, Manufacturing_plants, Distribution))

In [372]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,5, (Products, Distribution, Market))

In [373]:
# Plant Capacities
Capacities_i = np.random.randint(30,50,(Manufacturing_plants))
Capacities_j = np.random.randint(30,50,(Distribution))
Capacities_l = Supplier_cost = np.zeros((Products,Outsourced))
np.fill_diagonal(Capacities_l, random.randint(15,20))

In [374]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
Supplier_cost = np.zeros((Products, Outsourced))
np.fill_diagonal(Supplier_cost, random.randint(5,10))

In [375]:
# Cost of transporting product m from supplier to j
T_O_DC = np.zeros((Products, Outsourced, Distribution))
T_O_DC[0][0] = np.random.uniform(5,10,(Distribution))
T_O_DC[1][1] = np.random.uniform(5,10,(Distribution))

In [376]:
# Cost of shipping product m from supplier to k
T_O_MZ = np.zeros((Products, Outsourced, Market))
T_O_MZ[0][0] = np.random.uniform(5,10,(Market))
T_O_MZ[1][1] = np.random.uniform(5,10,(Market))

In [377]:
# Product volume 
volume = np.random.randint(2,5,(Products))

In [378]:
# unit cost of lost sales 
lost_sales = np.random.randint(150,200,(Market,Products))

In [379]:
## Model

In [380]:
grbModel = Model('synthetic')

In [381]:
a_i = np.ones(Manufacturing_plants)
b_j = np.ones(Distribution)

In [382]:
# Model Variables

x_i = grbModel.addVars(range(Manufacturing_plants), vtype = GRB.BINARY, obj = f_i) # opening manufacturing plant
x_j = grbModel.addVars(range(Distribution), vtype = GRB.BINARY, obj = f_j) # opening DC
U_km = grbModel.addVars(range(Market), range(Products), vtype = GRB.INTEGER, obj = lost_sales) # quantity lost sales
V_lm = grbModel.addVars(range(Products), range(Outsourced), vtype = GRB.INTEGER) # quantity products purchased from outsourcing
Q_im = grbModel.addVars(range(Products), range(Manufacturing_plants), vtype = GRB.INTEGER) # quantity produced
Y_ijm = grbModel.addVars(range(Products), range(Manufacturing_plants), range(Distribution), vtype = GRB.INTEGER) # shipping i -> j
Z_jkm = grbModel.addVars(range(Products), range(Distribution), range(Market), vtype = GRB.INTEGER) # shipping j -> k
T_ljm = grbModel.addVars(range(Products), range(Outsourced), range(Distribution), vtype = GRB.INTEGER) # shipping l -> j
T_lkm = grbModel.addVars(range(Products), range(Outsourced), range(Market), vtype = GRB.INTEGER) # shipping l -> k


In [383]:
# Model Constraints
# Network Flow
grbModel.addConstrs(quicksum(Y_ijm[m,i,j] for j in range(Distribution)) == Q_im[m,i] 
             for i in range(Manufacturing_plants) for m in range(Products))

grbModel.addConstrs(quicksum(Z_jkm[m,j,k] for k in range(Market)) == 
                      (quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) + 
                       quicksum(T_ljm[m,l,j] for l in range(Outsourced))) for j in range(Distribution) 
                      for m in range(Products))

grbModel.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) + quicksum(T_lkm[m,l,k] for l in range(Outsourced))) ==
                      (demand[m][k] - U_km[k,m]) 
                    for k in range(Market) for m in range(Products))


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>}

In [384]:
# Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
grbModel.addConstrs(V_lm[m,l] == quicksum(T_ljm[m,l,j] for j in range(Distribution)) + quicksum(T_lkm[m,l,k] for k in range(Market))
            for m in range(Products) for l in range(Outsourced))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>}

In [385]:
# Capacity Constraints
grbModel.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= a_i[i]*Capacities_i[i]*x_i[i] for i in range(Manufacturing_plants))
grbModel.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
            quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= b_j[j]*Capacities_j[j]*x_j[j] 
            for j in range(Distribution))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [390]:
# Objective
grb_expr = LinExpr()

# Cost of opening
OC_1 = 0
OC_2 = 0
for i in range(Manufacturing_plants):
    OC_1 += f_i[i]*x_i[i]
for j in range(Distribution):
    OC_2 += f_j[j]*x_j[j]
    
grb_expr += OC_1 + OC_2
ship_1 = 0
ship_2 = 0
ship_3 = 0
ship_4 = 0

# Shipment
for i in range(Manufacturing_plants):
    for j in range(Distribution):
        for m in range(Products):
            ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]
            
for j in range(Distribution):
    for k in range(Market):
        for m in range(Products):
            ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

for l in range(Outsourced):
    for j in range(Distribution):
        for m in range(Products):
            ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

for l in range(Outsourced):
    for k in range(Market):
        for m in range(Products):
            ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]

grb_expr += ship_1 + ship_2 + ship_3 + ship_4

# Production
pr_cost = 0
for i in range(Manufacturing_plants):
    for m in range(Products):
        pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]

grb_expr += pr_cost
# Buying from outsource cost
b_cost = 0
for l in range(Outsourced):
    for m in range(Products):
        b_cost += Supplier_cost[m][l]*V_lm[m,l]
grb_expr += b_cost

# Lost Sales
l_cost = 0
for k in range(Market):
    for m in range(Products):
        l_cost += lost_sales[k][m]*U_km[k,m]

grb_expr += l_cost
        

In [391]:
grbModel.setObjective(grb_expr, GRB.MINIMIZE)

In [392]:
grbModel.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 27 rows, 85 columns and 177 nonzeros
Model fingerprint: 0x56e08555
Variable types: 0 continuous, 85 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+01]

Loaded MIP start from previous solve with objective 0


Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [337]:
U_klm

NameError: name 'U_klm' is not defined